In [43]:
%reset -f

In [44]:

# coding: utf-8

# In[110]:

from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances


import pandas as pd
get_ipython().magic('matplotlib')


# In[111]:

df_all=pd.read_csv('C:/binary_2016_11_14.csv')
#df=pd.read_csv('C:/binary_2016_11_14.csv')

df_all = df_all[df_all['SEXO']==1]
msk = np.random.rand(len(df_all)) < 0.8
df, df_test = df_all[msk].copy(deep = True), df_all[~msk].copy(deep = True)
df = df.reset_index()
print(df.shape)
"""
msk = np.random.rand(len(df_all)) < 0.8

df= df_all[msk]
print("Longitud",len(df))

df_test = df_all[~msk]
print("Longitud2",len(df_test))
"""


# In[112]:

to_del = ['entity_id','sampleType','FECHA_NACIMIENTO','MUNICIPIO_RESIDENCIA',
          'MUNICIPIO_NACIMIENTO','PROVINCIA_RESIDENCIA','PAIS_NACIMIENTO', 'CALC_AVG_HEIGHT','CALC_AVG_PESO','CALC_AVG_CINTURA','CALC_AVG_CADERA' 
         ]
"""
to_del.extend(['ENFERMEDADES_DIABETES','ENFERMEDADES_HTA',
              'ENFERMEDADES_HIPERCOLESTEROLEMIA','ENFERMEDADES_ICTUS',
              'ENFERMEDADES_INFARTO','ENFERMEDADES_ANGINA'
              ])
"""

print (to_del)
print ("Enfermedades HTA",np.sum(df['ENFERMEDADES_HTA'] == 1))

#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
print ("df_2",df_2.shape)

# Filter complete null columns
cols = np.where((np.sum(df_2.isnull(), axis=0).values) == df_2.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2.columns if c not in df_2.columns[cols]]
df_3 = df_2[filt_cols]
print ("df_3",df_3.shape)

#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)

scaler = preprocessing.MinMaxScaler().fit(df_4)
data = scaler.transform(df_4)
data2 = df_4
print ("data",data.shape)

Using matplotlib backend: Qt5Agg
(4414, 353)
['entity_id', 'sampleType', 'FECHA_NACIMIENTO', 'MUNICIPIO_RESIDENCIA', 'MUNICIPIO_NACIMIENTO', 'PROVINCIA_RESIDENCIA', 'PAIS_NACIMIENTO', 'CALC_AVG_HEIGHT', 'CALC_AVG_PESO', 'CALC_AVG_CINTURA', 'CALC_AVG_CADERA']
Enfermedades HTA 865
df_2 (4414, 342)
[45 46 47 48 58 59]
df_3 (4414, 336)
df_4 (4414, 336)
data (4414, 336)


#### PCA

In [45]:
#### Feat sel with pca(0.8, 0.9) explained var 
thrd = 0.8
total = 0
pca = PCA().fit(data)
reduced_data = pca.transform(data)
for pca_comps,r in enumerate(pca.explained_variance_ratio_):
    if total > thrd:
        break
    total += r
print ("Num pca_comps per >", thrd,"ratio:", pca_comps, total)
print ("Explained variance first 2 components",pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])
print (pca.n_components_)
print (np.sum(pca.explained_variance_ratio_[:2]))

print ("PCA+K-means:", pca_comps)



Num pca_comps per > 0.8 ratio: 46 0.803501546849
Explained variance first 2 components 0.144404473231
336
0.144404473231
PCA+K-means: 46


In [46]:
n_clusters = 2
pca = PCA().fit(data)


plt.figure()
plt.plot(range(len(pca.explained_variance_ratio_)), np.cumsum(pca.explained_variance_ratio_))
plt.axvline(pca_comps, color="red")  
plt.ylim(0.0,1.1)
plt.show()

reduced_data = pca.transform(data)
#print ("Reduced data: ",reduced_data.shape)
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
kmeans.fit(reduced_data[:,:2])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [47]:
ar1 = df_3[kmeans.labels_==0]
print(type(df_3))
print(df_3.shape)
print(type(kmeans.labels_))
print(kmeans.labels_[0])
count=0
#newar = {'clusters': "cluster9"}
print("Example",df.loc[0,'ENFERMEDADES_DIABETES_T2DM'])
newar = pd.DataFrame()
for x in df_3.index: 
    #print("Count",count)
    newar.set_value(count,'clusters','{}{}'.format('clusters',kmeans.labels_[count]))
    #newar.set_value(count,'diabetes','{}{}'.format('diabetes',df.loc[count,'ENFERMEDADES_DIABETES_T2DM']))
    #newar.set_value(count,'HTA','{}{}'.format('HTA',df.loc[count,'ENFERMEDADES_HTA']))
    #newar.set_value(count,'col','{}{}'.format('col',df.loc[count,'ENFERMEDADES_HIPERCOLESTEROLEMIA']))
    #newar.set_value(count,'infarto','{}{}'.format('infarto',df.loc[count,'ENFERMEDADES_INFARTO']))
    #newar.set_value(count,'angina','{}{}'.format('angina',df.loc[count,'ENFERMEDADES_ANGINA']))
    #newar.set_value(count,'ictus','{}{}'.format('ictus',df.loc[count,'ENFERMEDADES_ICTUS']))
    if (df.loc[count,'ENFERMEDADES_DIABETES_T2DM']==0 and df.loc[count,'ENFERMEDADES_HIPERCOLESTEROLEMIA']==0 and df.loc[count,'ENFERMEDADES_HTA']==0 and df.loc[count,'ENFERMEDADES_ICTUS']==0 and df.loc[count,'ENFERMEDADES_ANGINA']==0 and df.loc[count,'ENFERMEDADES_INFARTO']==0):newar.set_value(count,'tipo','sano')
    else:
        if (df.loc[count,'ENFERMEDADES_DIABETES_T2DM']==0): newar.set_value(count,'tipo','nodiabetes')
        else:
            if (df.loc[count,'ENFERMEDADES_DIABETES_T2DM']==1 and df.loc[count,'ENFERMEDADES_HIPERCOLESTEROLEMIA']==0 and df.loc[count,'ENFERMEDADES_HTA']==0 and df.loc[count,'ENFERMEDADES_ICTUS']==0 and df.loc[count,'ENFERMEDADES_ANGINA']==0 and df.loc[count,'ENFERMEDADES_INFARTO']==0):newar.set_value(count,'tipo','diabetes')
            else:newar.set_value(count,'tipo','diabetescomplejo')

    if(df.loc[count,'BMI']<18.5): newar.set_value(count,'weight','underweight')
    else:
        if(df.loc[count,'BMI']<24.9): newar.set_value(count,'weight','normalweight')
        else:
            if (df.loc[count,'BMI']<29.9): newar.set_value(count,'weight','overweight')
            else:
                if (df.loc[count,'BMI']>29.8): newar.set_value(count,'weight','obese')
    if(df.loc[count,'EDAD_ANOS']<50): newar.set_value(count,'age','40s')
    else:
        if(df.loc[count,'EDAD_ANOS']<60): newar.set_value(count,'age','50s')
        else:
            if (df.loc[count,'EDAD_ANOS']<70): newar.set_value(count,'age','60s')
    if(df.loc[count,'ACTIVIDAD_FISICA_LIBRE_DEPORTE_REGULAR_0']==1): newar.set_value(count,'sport','sport')
    if(df.loc[count,'ACTIVIDAD_FISICA_LIBRE_DEPORTE_REGULAR_1']==1): newar.set_value(count,'sport','nosport')
    if(df.loc[count,'PREDIMED_SCORE']<5): newar.set_value(count,'Diet','DietVeryLow')
    else:
        if(df.loc[count,'PREDIMED_SCORE']<8): newar.set_value(count,'Diet','DietLow')
        else:
            if (df.loc[count,'PREDIMED_SCORE']<11): newar.set_value(count,'Diet','DietAverage')
            else:
                newar.set_value(count,'Diet','DietHigh') 
    #newar[count,'diabetes']='{}{}'.format('diabetes',df.loc[count,'ENFERMEDADES_DIABETES_T2DM'])
    #print("newwar",newar[count,'clusters'])
    count=count+1
#newar2  = pd.Series(newar,index=newar.keys())
print("count",count)
print("newarshape",newar.shape)
print("Fila: ",newar.head(2))
newar_ok=newar.values.tolist()
newar_ok_copy=newar_ok
"""
ar1_ok=ar1.values.tolist()
print(len(ar1_ok))
print(type(ar1))
ar2 = df[kmeans.labels_==1]
print(ar2.shape)
"""

<class 'pandas.core.frame.DataFrame'>
(4414, 336)
<class 'numpy.ndarray'>
1
Example 0.0
count 4414
newarshape (4414, 6)
Fila:      clusters        tipo        weight  age    sport         Diet
0  clusters1  nodiabetes  normalweight  40s  nosport  DietAverage
1  clusters0        sano         obese  50s      NaN  DietAverage


'\nar1_ok=ar1.values.tolist()\nprint(len(ar1_ok))\nprint(type(ar1))\nar2 = df[kmeans.labels_==1]\nprint(ar2.shape)\n'

In [49]:
print(newar_ok[0:5])
print("Longitud inicial: ",len(newar_ok))
for x in range(len(newar_ok)): 
    #print("X", x, ",",newar_ok[x])
    astring=newar_ok[x]
    y=0
    for y in range(len(astring),0,-1):
        #print("y",y-1,":",astring[y-1])
        if(str(astring[y-1]).endswith('nan')):
            newar_ok[x].remove(astring[y-1])
            #print("Removed",newar_ok[x])
    #if(len(newar_ok[x])==1):
    #print("Final: ", x, ",",newar_ok[x])

        #newar_ok=remove_element(newar_ok,x)
    #print(newar_ok[x])    
    #if (newar_ok[x].endswith('nan')):
        #newar_ok.pop(x)
    #print("Final2: ", x, ",",newar_ok[x])
    
for x in range (len(newar_ok),0,-1):
    if(len(newar_ok[x-1])==1):
        #print('hola')
        del newar_ok[x-1]
    
print("**",newar_ok[0:5])
print("Longitud final: ",len(newar_ok))

[['clusters1', 'nodiabetes', 'normalweight', '40s', 'nosport', 'DietAverage'], ['clusters0', 'sano', 'obese', '50s', 'DietAverage'], ['clusters1', 'diabetescomplejo', 'obese', '40s', 'nosport', 'DietAverage'], ['clusters1', 'diabetescomplejo', 'overweight', '50s', 'DietLow'], ['clusters1', 'diabetescomplejo', 'overweight', '40s', 'nosport', 'DietAverage']]
Longitud inicial:  4414
** [['clusters1', 'nodiabetes', 'normalweight', '40s', 'nosport', 'DietAverage'], ['clusters0', 'sano', 'obese', '50s', 'DietAverage'], ['clusters1', 'diabetescomplejo', 'obese', '40s', 'nosport', 'DietAverage'], ['clusters1', 'diabetescomplejo', 'overweight', '50s', 'DietLow'], ['clusters1', 'diabetescomplejo', 'overweight', '40s', 'nosport', 'DietAverage']]
Longitud final:  4414


#### Association rules

In [50]:
def load_dataset():
    "Load the sample dataset."
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return list(map(frozenset, c1))


def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    #print("longitud",len(list(dataset)))
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(list(dataset)))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data


def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList


def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = list(map(set, dataset))
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1

    return L, support_data

def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print ("freqSet", freqSet, 'H1', H1)
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules


def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            #print (freqSet - conseq, '--->', conseq, 'conf:', conf)
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H


def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [51]:
#dataset2=ar1.tolist()
dataset=newar_ok
"""
#for x in dataset2: print(x)
C1=createC1(dataset2)
#print("C1",C1)
D=map(set,dataset2)
L1,support_data=scanD(dataset2,C1,0.5)
print("L1",L1)
print("zeroo",dataset2[0][0])
print(type(dataset2))

#out=aprioriGen(L1,2)
#print("out",out)
#apriori(dataset)
#print("L",L)
#L,support_data=apriori(dataset,minsupport=0.5)
#print("L_",L)
"""
L,support_data=apriori(list(dataset),minsupport=0.001)
#for z in support_data: print("z",z)
#print("support_data",support_data)
rules=generateRules(L,support_data,min_confidence=0.7)
print("****Rules")
for z in rules: 
    #print(type(z))
    #print(z)
    if 'cluster' in str(z):
        print(z)
#print("*****Rules: ",rules)

****Rules
(frozenset({'40s'}), frozenset({'clusters1'}), 0.8803952158086323)
(frozenset({'DietLow'}), frozenset({'clusters1'}), 0.7212364052661705)
(frozenset({'normalweight'}), frozenset({'clusters1'}), 0.7016293279022403)
(frozenset({'DietVeryLow'}), frozenset({'clusters1'}), 0.7782426778242678)
(frozenset({'sport'}), frozenset({'clusters1'}), 0.8181818181818182)
(frozenset({'diabetescomplejo'}), frozenset({'clusters1'}), 0.743086978381096)
(frozenset({'diabetes', '40s'}), frozenset({'obese', 'clusters1'}), 0.7142857142857142)
(frozenset({'nosport', 'DietVeryLow', 'sano'}), frozenset({'40s', 'clusters1'}), 0.7199999999999999)
(frozenset({'diabetescomplejo', '40s', 'normalweight', 'DietAverage'}), frozenset({'nosport', 'clusters1'}), 0.7345132743362831)
(frozenset({'nosport', 'diabetescomplejo', 'DietVeryLow', 'overweight'}), frozenset({'40s', 'clusters1'}), 0.787878787878788)
(frozenset({'DietHigh', 'clusters0', '40s', 'sano'}), frozenset({'nosport', 'overweight'}), 0.714285714285714

In [52]:

newar_ok=newar.values.tolist()
rulesArrived = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
rulesOK = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for x in df_3.index:
    ruleindex=-1
    if (("40s" in newar_ok[x])):
        ruleindex=0
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("sport" in newar_ok[x])): 
        ruleindex=1
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x])and ("40s" in newar_ok[x]) and ("nodiabetes" in newar_ok[x])and ("overweight" in newar_ok[x])): 
        ruleindex=2
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x])and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietVeryLow" in newar_ok[x])): 
        ruleindex=3
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x]) and ("normalweight" in newar_ok[x]) and ("60s" in newar_ok[x]) and ("sano" in newar_ok[x])): 
        ruleindex=4
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters1" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("40s" in newar_ok[x]) and ("normalweight" in newar_ok[x]) and ("DietAverage" in newar_ok[x])): 
        ruleindex=5
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x])):
        ruleindex=6
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("nosport" in newar_ok[x]) and ("diabetescomplejo" in newar_ok[x]) and ("DietVeryLow" in newar_ok[x]) and ("overweight" in newar_ok[x])): 
        ruleindex=7
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("sport" in newar_ok[x])):
        ruleindex=8
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x])and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetes" in newar_ok[x]) and ("40s" in newar_ok[x])):
        ruleindex=9
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])and ("obese" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x]) and ("diabetescomplejo" in newar_ok[x]) and ("40s" in newar_ok[x]) and ("normalweight" in newar_ok[x])): 
        ruleindex=10
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietLow" in newar_ok[x])):
        ruleindex=11
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("DietVeryLow" in newar_ok[x]) and ("overweight" in newar_ok[x])): 
        ruleindex=12
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1

for y in range(0,13,1):
    print("y",y,rulesArrived[y], rulesOK[y])


y 0 1923 230
y 1 11 2
y 2 19 2
y 3 239 53
y 4 7 2
y 5 113 6
y 6 1989 511
y 7 33 1
y 8 6 0
y 9 7 0
y 10 31 4
y 11 1747 487
y 12 63 1


# Test rules

In [61]:
print(df_test.shape)
#df_test = df_test.reset_index()
# In[112]:

to_del = ['entity_id','sampleType','FECHA_NACIMIENTO','MUNICIPIO_RESIDENCIA',
          'MUNICIPIO_NACIMIENTO','PROVINCIA_RESIDENCIA','PAIS_NACIMIENTO', 'CALC_AVG_HEIGHT','CALC_AVG_PESO','CALC_AVG_CINTURA','CALC_AVG_CADERA' 
         ]
"""
to_del.extend(['ENFERMEDADES_DIABETES','ENFERMEDADES_HTA',
              'ENFERMEDADES_HIPERCOLESTEROLEMIA','ENFERMEDADES_ICTUS',
              'ENFERMEDADES_INFARTO','ENFERMEDADES_ANGINA'
              ])
"""

#Filter_selected cols
filtered_cols = [c for c in df_test.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df_test[filtered_cols]
print ("df_2",df_2.shape)

# Filter complete null columns
cols = np.where((np.sum(df_2.isnull(), axis=0).values) == df_2.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2.columns if c not in df_2.columns[cols]]
df_3 = df_2[filt_cols]
print ("df_3",df_3.shape)

#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)

scaler = preprocessing.MinMaxScaler().fit(df_4)
data = scaler.transform(df_4)
data2 = df_4
print ("data",data.shape)

thrd = 0.8
total = 0
pca = PCA().fit(data)
reduced_data = pca.transform(data)
for pca_comps,r in enumerate(pca.explained_variance_ratio_):
    if total > thrd:
        break
    total += r
print ("Num pca_comps per >", thrd,"ratio:", pca_comps, total)
print ("Explained variance first 2 components",pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])
print (pca.n_components_)
print (np.sum(pca.explained_variance_ratio_[:2]))

print ("PCA+K-means:", pca_comps)

n_clusters = 2
pca = PCA().fit(data)


reduced_data = pca.transform(data)

kmeans.predict(reduced_data[:,:2])
print(kmeans.labels_)
kmeans.labels_.shape

newar = pd.DataFrame()
count=0
for x in df_3.index: 
    #print("Count",count)
    newar.set_value(count,'clusters','{}{}'.format('clusters',kmeans.labels_[count])) 
    if (df_test.loc[count,'ENFERMEDADES_DIABETES_T2DM']==0 and df_test.loc[count,'ENFERMEDADES_HIPERCOLESTEROLEMIA']==0 and df_test.loc[count,'ENFERMEDADES_HTA']==0 and df_test.loc[count,'ENFERMEDADES_ICTUS']==0 and df_test.loc[count,'ENFERMEDADES_ANGINA']==0 and df_test.loc[count,'ENFERMEDADES_INFARTO']==0):newar.set_value(count,'tipo','sano')
    else:
        if (df_test.loc[count,'ENFERMEDADES_DIABETES_T2DM']==0): newar.set_value(count,'tipo','nodiabetes')
        else:
            if (df_test.loc[count,'ENFERMEDADES_DIABETES_T2DM']==1 and df_test.loc[count,'ENFERMEDADES_HIPERCOLESTEROLEMIA']==0 and df_test.loc[count,'ENFERMEDADES_HTA']==0 and df_test.loc[count,'ENFERMEDADES_ICTUS']==0 and df_test.loc[count,'ENFERMEDADES_ANGINA']==0 and df_test.loc[count,'ENFERMEDADES_INFARTO']==0):newar.set_value(count,'tipo','diabetes')
            else:newar.set_value(count,'tipo','diabetescomplejo')
    if(df_test.loc[count,'BMI']<18.5): newar.set_value(count,'weight','underweight')
    else:
        if(df_test.loc[count,'BMI']<24.9): newar.set_value(count,'weight','normalweight')
        else:
            if (df_test.loc[count,'BMI']<29.9): newar.set_value(count,'weight','overweight')
            else:
                if (df_test.loc[count,'BMI']>29.8): newar.set_value(count,'weight','obese')
    if(df_test.loc[count,'EDAD_ANOS']<50): newar.set_value(count,'age','40s')
    else:
        if(df_test.loc[count,'EDAD_ANOS']<60): newar.set_value(count,'age','50s')
        else:
            if (df_test.loc[count,'EDAD_ANOS']<70): newar.set_value(count,'age','60s')
    if(df.loc[count,'ACTIVIDAD_FISICA_LIBRE_DEPORTE_REGULAR_0']==1): newar.set_value(count,'sport','sport')
    if(df.loc[count,'ACTIVIDAD_FISICA_LIBRE_DEPORTE_REGULAR_1']==1): newar.set_value(count,'sport','nosport')
    if(df.loc[count,'PREDIMED_SCORE']<5): newar.set_value(count,'Diet','DietVeryLow')
    else:
        if(df.loc[count,'PREDIMED_SCORE']<8): newar.set_value(count,'Diet','DietLow')
        else:
            if (df.loc[count,'PREDIMED_SCORE']<11): newar.set_value(count,'Diet','DietAverage')
            else:
                newar.set_value(count,'Diet','DietHigh') 
    count=count+1
print("count",count)
print("newarshape",newar.shape)
print("Fila: ",newar.head(2))
newar_ok=newar.values.tolist()
newar_ok_copy=newar_ok

for x in range(len(newar_ok)): 
    #print("X", x, ",",newar_ok[x])
    astring=newar_ok[x]
    y=0
    for y in range(len(astring),0,-1):
        #print("y",y-1,":",astring[y-1])
        if(str(astring[y-1]).endswith('nan')):
            newar_ok[x].remove(astring[y-1])
            #print("Removed",newar_ok[x])
    #if(len(newar_ok[x])==1):
    #print("Final: ", x, ",",newar_ok[x])

        #newar_ok=remove_element(newar_ok,x)
    #print(newar_ok[x])    
    #if (newar_ok[x].endswith('nan')):
        #newar_ok.pop(x)
    #print("Final2: ", x, ",",newar_ok[x])
    
for x in range (len(newar_ok),0,-1):
    if(len(newar_ok[x-1])==1):
        #print('hola')
        del newar_ok[x-1]
    
print("**",newar_ok[0:5])
print("Longitud final: ",len(newar_ok))

newar_ok=newar.values.tolist()
rulesArrived = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
rulesOK = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for x in df_3.index:
    ruleindex=-1
    if (("40s" in newar_ok[x])):
        ruleindex=0
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("sport" in newar_ok[x])): 
        ruleindex=1
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x])and ("40s" in newar_ok[x]) and ("nodiabetes" in newar_ok[x])and ("overweight" in newar_ok[x])): 
        ruleindex=2
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x])and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietVeryLow" in newar_ok[x])): 
        ruleindex=3
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x]) and ("normalweight" in newar_ok[x]) and ("60s" in newar_ok[x]) and ("sano" in newar_ok[x])): 
        ruleindex=4
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters1" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("40s" in newar_ok[x]) and ("normalweight" in newar_ok[x]) and ("DietAverage" in newar_ok[x])): 
        ruleindex=5
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x])):
        ruleindex=6
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("nosport" in newar_ok[x]) and ("diabetescomplejo" in newar_ok[x]) and ("DietVeryLow" in newar_ok[x]) and ("overweight" in newar_ok[x])): 
        ruleindex=7
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("sport" in newar_ok[x])):
        ruleindex=8
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x])and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetes" in newar_ok[x]) and ("40s" in newar_ok[x])):
        ruleindex=9
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])and ("obese" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietHigh" in newar_ok[x]) and ("diabetescomplejo" in newar_ok[x]) and ("40s" in newar_ok[x]) and ("normalweight" in newar_ok[x])): 
        ruleindex=10
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("nosport" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("DietLow" in newar_ok[x])):
        ruleindex=11
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1
    if (("diabetescomplejo" in newar_ok[x]) and ("DietVeryLow" in newar_ok[x]) and ("overweight" in newar_ok[x])): 
        ruleindex=12
        rulesArrived[ruleindex]=rulesArrived[ruleindex]+1
        if (("40s" in newar_ok[x]) and ("clusters0" in newar_ok[x])): rulesOK[ruleindex]=rulesOK[ruleindex]+1


for y in range(0,13,1):
    print("y",y,rulesArrived[y], rulesOK[y])

(1130, 354)
df_2 (1130, 343)
[46 47 48 49 59 60]
df_3 (1130, 337)
df_4 (1130, 337)
data (1130, 337)
Num pca_comps per > 0.8 ratio: 44 0.802450643395
Explained variance first 2 components 0.146532196687
337
0.146532196687
PCA+K-means: 44
[1 0 1 ..., 1 0 0]
count 1130
newarshape (1130, 6)
Fila:      clusters              tipo        weight  age    sport         Diet
0  clusters1  diabetescomplejo         obese  40s  nosport  DietAverage
1  clusters0  diabetescomplejo  normalweight  40s      NaN  DietAverage
** [['clusters1', 'diabetescomplejo', 'obese', '40s', 'nosport', 'DietAverage'], ['clusters0', 'diabetescomplejo', 'normalweight', '40s', 'DietAverage'], ['clusters1', 'nodiabetes', 'overweight', '60s', 'nosport', 'DietAverage'], ['clusters1', 'nodiabetes', 'overweight', '50s', 'DietLow'], ['clusters1', 'diabetescomplejo', 'overweight', '40s', 'nosport', 'DietAverage']]
Longitud final:  1130
y 0 476 155
y 1 3 0
y 2 4 0
y 3 58 17
y 4 0 0
y 5 29 8
y 6 524 169
y 7 4 0
y 8 0 0
y 9 0 0
y 1

In [62]:
for y in range(0,13,1):
    if (rulesArrived[y]!=0):
        print("y",y,rulesOK[y]/rulesArrived[y])

y 0 0.32563025210084034
y 1 0.0
y 2 0.0
y 3 0.29310344827586204
y 5 0.27586206896551724
y 6 0.32251908396946566
y 7 0.0
y 10 0.25
y 11 0.24770642201834864
y 12 0.09090909090909091
